In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import requests
from config import rapid_api_key
from config import rapidapi_host

In [12]:
# os.getcwd()
# os.chdir('C:\\gitlab\\covid19')

In [ ]:
url = "https://covid-19-statistics.p.rapidapi.com/reports/total"

querystring = {"date":"2020-04-07"}

headers = {
    rapid_api_key,
    rapidapi_host
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)